In [1]:
print('HIHI')

HIHI


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [4]:
# extracted_data
extracted_data=load_pdf_file(data='data/')

In [5]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
# text_chunks
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 135


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\LUU THE TRUNG\AppData\Local\Temp\ipykernel_18156\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\LUU THE TRUNG\anaconda3\envs\healthybot_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
query_result = embeddings.embed_query("Xin chào")
print("Length", len(query_result))

Length 384


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

## Use vector db Pinecone

In [12]:
# import os
# PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
# OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [13]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec


# pc = Pinecone(api_key=PINECONE_API_KEY)

# index_name = "healthbot"


# pc.create_index(
#     name=index_name,
#     dimension=384, 
#     metric="cosine", 
#     spec=ServerlessSpec(
#         cloud="aws", 
#         region="us-east-1"
#     ) 
# ) 

In [14]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
# from langchain.vectorstores import Pinecone

# docsearch = Pinecone.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings, 
# )

In [15]:
# Load Existing index 

# from langchain.vectorstores import Pinecone
# # Embed each chunk and upsert the embeddings into your Pinecone index.
# docsearch = Pinecone.from_existing_index(
#     index_name=index_name,
#     embedding=embeddings
# )

In [16]:
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [17]:
# retrieved_docs = retriever.invoke("Bé bị sốt, phải làm sao?")

In [18]:
# retrieved_docs

## Use vector DB Faiss

In [19]:
from langchain_community.vectorstores import FAISS

vector_db_path = 'vector_db/db_faiss'

def create_db():
    db = FAISS.from_documents(text_chunks, embeddings)
    db.save_local(vector_db_path)
    return db

create_db()

## Use OpenAI

In [20]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=200)

In [21]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.prompts import ChatPromptTemplate


# system_prompt = (
#     "Bạn là trợ lý ảo có nhiệm vụ trả lời các câu hỏi về sức khỏe. "
#     "Sử dụng các thông tin trong tài liệu để trả lời câu hỏi. "
#     "Nếu không biết câu trả lời, hãy nói rằng bạn không biết. "
#     "Sử dụng ngôn ngữ tự nhiên và trả lời ngắn gọn."
#     "\n\n"
#     "{context}"
# )


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [22]:
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [23]:
# response = rag_chain.invoke({"input": "Bé bị sốt, phải làm sao?"})
# print(response["answer"])

## Use HuggingFace 

In [24]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [25]:
model_file = 'models/vinallama-7b-chat_q5_0.gguf'

def load_model(model_file):
    llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=1024,
        temperature=0.01
    )
    return llm

### Test model

In [26]:
# def creart_prompt(template):
#     prompt = PromptTemplate(template=template, input_variables=['question'])
#     return prompt

# def create_simple_chain(prompt, llm):
#     chain = LLMChain(prompt=prompt, llm=llm)
#     return chain

In [27]:
# template = '''
# <|im_start|>system
# Bạn là trợ lý ảo hữu ích. Hãy trả lời người dùng chính xác. Sử dụng ngôn ngữ tự nhiên và trả lời ngắn gọn.
# <|im_end|>
# <|im_start|>user
# {question}<|im_end|>
# <|im_start|>assistant
# '''

# prompt = creart_prompt(template)
# llm = load_model(model_file)
# chain = create_simple_chain(prompt, llm)

# response = chain.invoke({"question": "Ai là tổng thống Mỹ năm 2024?"})
# print(response)

In [28]:
def creart_prompt(template):
    prompt = PromptTemplate(template=template, input_variables=['context', 'question'])
    return prompt

def create_qa_chain(prompt, llm, db):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k":2}),
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

def read_vector_db():
    db = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)
    return db

In [29]:
template = '''
<|im_start|>system
Bạn là trợ lý ảo có nhiệm vụ trả lời các câu hỏi về sức khỏe. Sử dụng các thông tin trong tài liệu để trả lời câu hỏi. Nếu không biết câu trả lời, hãy nói rằng bạn không biết. Sử dụng ngôn ngữ tự nhiên và trả lời ngắn gọn.
{context}<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
'''

In [30]:
llm = load_model(model_file)
prompt = creart_prompt(template)
db = read_vector_db()

llm_chain = create_qa_chain(prompt, llm, db)

response = llm_chain.invoke({"query": "Bé bị sốt, phải làm sao?"})
print(response)

{'query': 'Bé bị sốt, phải làm sao?', 'result': 'Nếu trẻ dưới 6 tháng tuổi và bị sốt, bạn nên gọi trợ giúp y tế nếu trẻ:\n- Bị cứng cổ.\n- Có các vấn đề khác ngoài sốt như nôn mửa, cứng tai, phát ban hoặc co giật.\n\nĐối với trẻ trên 6 tháng tuổi và bị sốt kéo dài hơn 48 giờ, hãy gọi bác sĩ ngay nếu chúng có các triệu chứng sau đây :\n- Tiêu chảy kéo dài hơn 24 giờ.\n- Bị tiêu chảy và nôn mửa.\n- Bị tiêu chảy có phân màu đen hoặc có máu trong phân kèm theo sốt trên 38,5°C.\n- Rất yếu ớt và từ chối ăn hoặc uống.\n- Có dấu hiệu mất nước như mắt trũng, da khô, miệng khát, đi tiểu ít hoặc nước tiểu vàng đậm.'}
